In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sys
sys.path.append('..')
import pytpc
from pytpc.constants import *
from scipy.ndimage.filters import gaussian_filter
from math import sin, cos

In [ ]:
tilt = 7*degrees
clock = 12.5  # MHz
emag = 15e3
bmag = 0.5691
ef = np.array([0, 0, emag])
bfield_orig = np.array([0, 0, bmag])
trans = np.array([[1, 0, 0],
                  [0, cos(-tilt), -sin(-tilt)],
                  [0, sin(-tilt), cos(-tilt)]])
bf = np.dot(trans, bfield_orig)
vd = pytpc.simulation.drift_velocity_vector(-3.32, emag, bmag, tilt)

In [ ]:
proj = pytpc.Particle(4, 2, 1.5, position=(0, 0, 1.), azimuth=0, polar=pi+tilt)
target = pytpc.Particle(4, 2, 0)
heco = pytpc.gases.HeCO2Gas(150.)

simres = pytpc.simulation.simulate_elastic_scattering_track(proj, target, heco, ef, bf, 4, pi/2, 0)
simres.x *= 1000
simres.y *= 1000
simres.z *= 1000
# simres['pos'][:, 2] = 1000 - simres['pos'][:, 2]

In [ ]:
evt = pytpc.evtdata.make_event(simres[['x', 'y', 'z']].values, simres.de.values, clock, vd, 40.2, proj.mass_num, 334.)

In [ ]:
pytpc.pad_plot(evt.hits());

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
plt.plot(evt.traces['data'].sum(0), 'k--')
for x in evt.traces['data']:
    plt.plot(x)

In [ ]:
chfig = pytpc.chamber_plot(evt.xyzs(vd, clock), zscale='dist')
chfig.axes[0].view_init(elev=0, azim=50)